# Домашнее задание №2
В данном задании предлагается решить [контест](https://www.kaggle.com/competitions/vk-edu-bmstu-asr-competition/overview) на платформе Kaggle.
Основная цель данного задания состоит в том, чтобы студенты ознакомились с зоопарком открытых моделей для решения задачи распознавания речи, а так же попрактиковались в подходах по постпроцессингу распознанных гипотез для улучшения качества ASR.

Разрешается пользоваться всеми предобученными моделями.

**Строго запрещено размечать данные вручную!**

## Загрузка данных и инициализация модели

In [1]:
# https://drive.google.com/file/d/1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG/view?usp=sharing # ссылка для ручной загрузки данных

In [ ]:
!pip install gdown

In [3]:
!gdown 1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG && tar -xf kaggle.tar

Downloading...
From (original): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG
From (redirected): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG&confirm=t&uuid=babc9595-5382-4608-a9aa-292b0ba835e0
To: /kaggle/working/kaggle.tar
100%|████████████████████████████████████████| 138M/138M [00:02<00:00, 63.5MB/s]


In [4]:
!ls -l kaggle/wav | wc -l

643


In [ ]:
!pip install num2words pyannote.audio silero_vad

In [6]:
import os
import soundfile as sf
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np
import string
import re
from num2words import num2words
from pyannote.audio import Pipeline

from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [8]:
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
processor.feature_extractor.return_attention_mask = True
model = WhisperForConditionalGeneration.from_pretrained(model_name).half().to(device)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

## Обработка отсутствия речи

In [9]:
# !huggingface-cli login

In [10]:
path_to_wavs = "kaggle/wav"
wav_files = os.listdir(path_to_wavs)

In [11]:
vad_pipeline = Pipeline.from_pretrained("pyannote/voice-activity-detection", use_auth_token="")
vad_pipeline.to(torch.device("cuda"))

def has_speech(audio_path):
    vad_result = vad_pipeline(audio_path)
    speech_segments = vad_result.get_timeline().support()
    return len(speech_segments) > 0

files_with_silence = []
for wav_file in tqdm(wav_files, desc="Проверка наличия речи"):
    audio_path = os.path.join(path_to_wavs, wav_file)
    if not has_speech(audio_path):
        files_with_silence.append(wav_file)

config.yaml:   0%|          | 0.00/277 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.7.1, yours is 2.4.0. Bad things might happen unless you revert torch to 1.x.


Проверка наличия речи:   0%|          | 0/642 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
Проверка наличия речи: 100%|██████████| 642/642 [00:13<00:00, 46.96it/s]


In [12]:
from silero_vad import get_speech_timestamps, read_audio, load_silero_vad

def load_vad_model():
    model, _ = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=True)
    return model.to(torch.device("cuda"))

def has_speech2(audio_path, vad_model, sampling_rate=16000):
    wav = read_audio(audio_path, sampling_rate=sampling_rate).to(torch.device("cuda"))
    speech_timestamps = get_speech_timestamps(wav, vad_model, sampling_rate=sampling_rate)
    return len(speech_timestamps) > 0

vad_model = load_vad_model()
files_with_silence2 = []

for wav_file in tqdm(wav_files, desc="Проверка наличия речи"):
    audio_path = os.path.join(path_to_wavs, wav_file)
    if not has_speech2(audio_path, vad_model):
        files_with_silence2.append(wav_file)

/opt/conda/lib/python3.10/site-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip
Проверка наличия речи: 100%|██████████| 642/642 [01:18<00:00,  8.17it/s]


In [13]:
files_with_silences = set(files_with_silence + files_with_silence2)
files_with_silences

{'audio_0.wav',
 'audio_1.wav',
 'audio_100.wav',
 'audio_102.wav',
 'audio_103.wav',
 'audio_107.wav',
 'audio_12.wav',
 'audio_123.wav',
 'audio_127.wav',
 'audio_13.wav',
 'audio_14.wav',
 'audio_15.wav',
 'audio_16.wav',
 'audio_164.wav',
 'audio_168.wav',
 'audio_169.wav',
 'audio_173.wav',
 'audio_178.wav',
 'audio_23.wav',
 'audio_24.wav',
 'audio_254.wav',
 'audio_255.wav',
 'audio_262.wav',
 'audio_263.wav',
 'audio_286.wav',
 'audio_297.wav',
 'audio_30.wav',
 'audio_306.wav',
 'audio_309.wav',
 'audio_385.wav',
 'audio_39.wav',
 'audio_391.wav',
 'audio_392.wav',
 'audio_473.wav',
 'audio_490.wav',
 'audio_491.wav',
 'audio_492.wav',
 'audio_493.wav',
 'audio_497.wav',
 'audio_53.wav',
 'audio_54.wav',
 'audio_55.wav',
 'audio_56.wav',
 'audio_560.wav',
 'audio_561.wav',
 'audio_57.wav',
 'audio_58.wav',
 'audio_59.wav',
 'audio_593.wav',
 'audio_60.wav',
 'audio_61.wav',
 'audio_62.wav',
 'audio_63.wav',
 'audio_68.wav',
 'audio_69.wav',
 'audio_70.wav',
 'audio_71.wav',
 '

## Инференс модели

In [14]:
def batch_inference(model, processor, wav_files, batch_size, sampling_rate=16000, beam_size=3):
    results = {}

    forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")

    for i in tqdm(range(0, len(wav_files), batch_size), total=np.ceil(len(wav_files) / batch_size)):
        audio_paths = wav_files[i : i + batch_size]

        batch = []

        for path in audio_paths:
            audio, _ = sf.read(os.path.join(path_to_wavs, path))
            batch.append(audio)

        inputs = processor(batch, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
        x, x_masks = inputs["input_features"].half().to(device), inputs["attention_mask"].to(device)

        with torch.inference_mode():
          output_ids = model.generate(
              x,
              forced_decoder_ids=forced_decoder_ids,
              attention_mask=x_masks,
              num_beams=beam_size,
              length_penalty=1.0,
              early_stopping=True,
              output_scores=True,
          )

        transcriptions = processor.batch_decode(output_ids, skip_special_tokens=True)

        results.update(zip(audio_paths, transcriptions))

        del inputs, x, x_masks, output_ids, transcriptions
        torch.cuda.empty_cache()

    return results

In [15]:
wav_files = os.listdir(path_to_wavs)
wav_files_without_silence = [
    wav_file for wav_file in wav_files if wav_file not in files_with_silences
]

In [16]:
results = batch_inference(
    model = model,
    processor=processor,
    wav_files=wav_files_without_silence,
    batch_size=8,
    beam_size=3, # beam search
)

  0%|          | 0/71.0 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_scores` is. When `return_dict_in_generate` is not `True`, `output_scores` is ignored.
  warnings.warn(
100%|██████████| 71/71.0 [11:59<00:00, 10.14s/it]


## Постпроцессинг

In [17]:
import re
from num2words import num2words
import string

def postprocessing(data):
    """ Постобработка: замена знаков препинания на пробелы,
    приведение к нижнему регистру, преобразование чисел в текст,
    удаление тире, не являющихся частью слов. """
    translator = str.maketrans(string.punctuation.replace('-', '') + '«»…"""''',
                               ' ' * (len(string.punctuation.replace('-', '')) + len('«»…"""''')))

    processed_data = {}
    for filename, hypo in data.items():
        hypo = hypo.strip()
        hypo = hypo.translate(translator)
        hypo = hypo.lower()

        hypo = re.sub(r'(?<!\w)[\-–—]+(?!\w)', ' ', hypo)

        def replace_numbers_in_word(match):
            word = match.group()
            numbers_in_word = re.findall(r'\d+', word)
            for num in numbers_in_word:
                word = word.replace(num, num2words(int(num), lang='ru'))
            return word

        hypo = re.sub(r'\w*\d+\w*', replace_numbers_in_word, hypo)

        hypo = re.sub(r'\s+', ' ', hypo).strip()

        processed_data[filename] = hypo

    return processed_data

In [18]:
processed_results = postprocessing(results)

## Сохранение результатов

In [22]:
data = []

for filename, hypo in processed_results.items():
    file_id = int(re.findall(r'\d+', filename)[0])
    data.append({"id": file_id, "filename": filename, "hypo": hypo})

for filename in files_with_silences:
    file_id = int(re.findall(r'\d+', filename)[0])
    data.append({"id": file_id, "filename": filename, "hypo": " "})

df = pd.DataFrame(data)
df = df.sort_values(by="id").reset_index(drop=True)

df.to_csv("processed_results.csv", index=False, encoding="utf-8")